In [ ]:
import glob
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

plt.style.use("seaborn-v0_8-paper")
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# # Load data
# df = pd.read_csv("result/convergence_results.csv")
# h = df["h"].values
# err_mb = df["error_mb"].values
# err_mf = df["error_mf"].values

# # Setup plot
# plt.figure(figsize=(10, 7))

# # Plot Solvers
# plt.loglog(h, err_mb, "o-", linewidth=2, label="Matrix-Based", markersize=8)
# plt.loglog(h, err_mf, "s--", linewidth=2, label="Matrix-Free", markersize=8)

# # Add Reference Lines (Theoretical Order)
# # For FE Degree 2:
# # L2 Error should be O(h^(p+1)) = O(h^3)
# # H1 Error should be O(h^p)     = O(h^2)

# # We create a reference line aligned with the last data point
# ref_h = h
# # O(h^3) Reference
# ref_error_3 = ref_h**3
# plt.loglog(ref_h, ref_error_3, "k:", linewidth=1.5, label="Reference O($h^3$)")

# # O(h^2) Reference (optional, usually for H1 norm)
# ref_error_2 = ref_h**2
# plt.loglog(
#     ref_h, ref_error_2, "k-.", linewidth=1.0, alpha=0.5, label="Reference O($h^2$)"
# )

# # Formatting
# plt.title("Convergence Study: Advection-Diffusion-Reaction", fontsize=16)
# plt.xlabel("Grid Size ($h$)", fontsize=14)
# plt.ylabel("$L_2$ Error Norm", fontsize=14)
# plt.grid(True, which="both", ls="-", alpha=0.4)
# plt.legend(fontsize=12)

# # Invert X axis so smaller h is on the right?
# # Standard convention is smaller h on left, but loglog handles scale automatically.
# # Usually we want h decreasing from right to left or just standard increasing x.
# # Matplotlib default is increasing x (small h on left).

# plt.tight_layout()
# plt.savefig("./report/figures/convergence_plot.pdf", dpi=300)
# plt.show()

# # Print Convergence Rates
# print("\nCalculated Convergence Rates (Matrix-Free):")
# for i in range(1, len(h)):
#     rate = np.log(err_mf[i - 1] / err_mf[i]) / np.log(h[i - 1] / h[i])
#     print(f"Refinement {i + 2} -> {i + 3}: Rate = {rate:.4f}")

In [ ]:
# Read all CSV files
csv_files = glob.glob("result/complexity_*.csv")
# Combine all results
all_results = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    config_name = Path(csv_file).stem.replace("complexity_", "")
    df["config"] = config_name
    all_results.append(df)
results_df = pd.concat(all_results, ignore_index=True)
results_df.head()

In [ ]:
from itertools import cycle

import matplotlib.pyplot as plt

palette = sns.color_palette("colorblind")
colors = cycle(palette)

linestyles = cycle(["-", "--", "-.", ":"])
markers = cycle(["o", "s", "D", "^", "v", "P"])

fig = plt.figure(figsize=(10, 7))

for config in sorted(results_df["config"].unique()):
    config_data = results_df[results_df["config"] == config]
    mb = config_data[config_data["solver_type"] == "matrix_based"].sort_values("n_dofs")

    if not mb.empty:
        plt.plot(
            mb["n_dofs"],
            mb["total_time"],
            label=config,
            color=next(colors),
            linestyle=next(linestyles),
            marker=next(markers),
            linewidth=2,
            markersize=5,
        )

plt.xlabel("Degrees of Freedom")
plt.ylabel("Total Time (seconds)")
plt.title("Matrix-Based Solver: Time Complexity")

plt.xscale("log")
plt.yscale("log")

plt.legend(frameon=False, fontsize=9)
plt.grid(True, which="both", linestyle=":", alpha=0.3)
plt.savefig("./report/figures/complexity_matrix_based.pdf", dpi=300)
plt.tight_layout()
plt.show()

In [ ]:
from itertools import cycle

import matplotlib.pyplot as plt

palette = sns.color_palette("colorblind")
colors = cycle(palette)

linestyles = cycle(["-", "--", "-.", ":"])
markers = cycle(["o", "s", "D", "^", "v", "P"])

fig = plt.figure(figsize=(10, 7))

for config in sorted(results_df["config"].unique()):
    config_data = results_df[results_df["config"] == config]
    mb = config_data[config_data["solver_type"] == "matrix_free"].sort_values("n_dofs")

    if not mb.empty:
        plt.plot(
            mb["n_dofs"],
            mb["total_time"],
            label=config,
            color=next(colors),
            linestyle=next(linestyles),
            marker=next(markers),
            linewidth=2,
            markersize=5,
        )

plt.xlabel("Degrees of Freedom")
plt.ylabel("Total Time (seconds)")
plt.title("Matrix-Free Solver: Time Complexity")

plt.xscale("log")
plt.yscale("log")

plt.legend(frameon=False, fontsize=9)
plt.grid(True, which="both", linestyle=":", alpha=0.3)
plt.savefig("./report/figures/complexity_matrix_free.pdf", dpi=300)
plt.tight_layout()
plt.show()

In [ ]:
def format_dofs(n_dofs):
    """Format DoFs to human readable format (e.g., 67.1M, 16.8M, 1.1M)"""
    if n_dofs >= 1e6:
        return f"{n_dofs / 1e6:.1f}M"
    elif n_dofs >= 1e3:
        return f"{n_dofs / 1e3:.0f}K"
    else:
        return str(int(n_dofs))


def get_ideal_scaling(x_values, y_start, x_start=None):
    """Calculate ideal scaling line (linear speedup)"""
    if x_start is None:
        x_start = x_values[0]
    return y_start * x_start / x_values


COLOR_CYCLE = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
]
MARKERS = ["o", "s", "D", "^", "v", "p", "h"]

print("Helper functions defined!")

In [ ]:
# Load all CSV files
all_results = pd.read_csv("result/combined_all_results.csv")
pure_mpi = pd.read_csv("result/combined_pure_mpi.csv")
hybrid = pd.read_csv("result/combined_hybrid.csv")
weak = pd.read_csv("result/combined_weak.csv")

# Filter for strong scaling tests
strong_df = all_results[all_results["test_type"] == "strong_scaling"].copy()
pure_mpi_strong = pure_mpi[pure_mpi["test_type"] == "strong_scaling"].copy()

print(f"Loaded {len(all_results)} total results")
print(f"Strong scaling data: {len(strong_df)} rows")
print(f"Pure MPI strong scaling: {len(pure_mpi_strong)} rows")
print(f"\nUnique DoF values: {sorted(pure_mpi_strong['n_dofs'].unique())}")
print(f"Unique processor counts: {sorted(pure_mpi_strong['n_mpi'].unique())}")

In [ ]:
def plot_strong_scaling_all_solvers(df, save_path_prefix=None):
    """
    Create three separate figures showing strong scaling:
    (a) Total time for all solvers
    (b) Solve time for all solvers
    (c) Setup+Assemble time for all solvers
    """
    # Get unique DoF values sorted descending
    dof_values = sorted(df["n_dofs"].unique(), reverse=True)
    processors = sorted(df["n_mpi"].unique())

    plot_idx = 0
    legend_handles = []
    legend_labels = []

    # Collect data for all plots
    plot_data = []

    for solver_type in ["matrix_free", "matrix_based"]:
        solver_df = df[df["solver_type"] == solver_type]

        for i, n_dofs in enumerate(dof_values[:3]):  # Top 3 DoF values
            subset = solver_df[solver_df["n_dofs"] == n_dofs].sort_values("n_mpi")

            if len(subset) < 2:
                continue

            x = subset["n_mpi"].values
            abbrev = "mf" if solver_type == "matrix_free" else "mb"
            label = f"{abbrev} ({format_dofs(n_dofs)} DoFs)"

            color = COLOR_CYCLE[plot_idx % len(COLOR_CYCLE)]
            marker = MARKERS[plot_idx % len(MARKERS)]
            linestyle = "-" if solver_type == "matrix_free" else "-."

            y_total = subset["total_time_avg"].values
            y_solve = subset["solve_time_avg"].values
            y_setup = (
                subset["setup_time_avg"].values + subset["assembly_time_avg"].values
            )

            plot_data.append(
                {
                    "x": x,
                    "y_total": y_total,
                    "y_solve": y_solve,
                    "y_setup": y_setup,
                    "color": color,
                    "marker": marker,
                    "linestyle": linestyle,
                    "label": label,
                }
            )
            plot_idx += 1

    x_ideal = np.array(processors)

    # Plot 1: Total time
    fig1, ax1 = plt.subplots(figsize=(10, 7))
    for data in plot_data:
        (line,) = ax1.plot(
            data["x"],
            data["y_total"],
            marker=data["marker"],
            color=data["color"],
            linestyle=data["linestyle"],
            label=data["label"],
            markeredgecolor="black",
            markeredgewidth=0.5,
        )

    y_ref_total = df["total_time_avg"].max() * 0.5
    y_ideal = get_ideal_scaling(x_ideal, y_ref_total, x_ideal[0])
    ax1.plot(x_ideal, y_ideal, "k--", linewidth=2, label="Ideal scaling")

    ax1.set_yscale("log")
    ax1.set_xlabel("Number of processors")
    ax1.set_ylabel("total time (s)")
    ax1.set_title("Strong scaling for total time for all the solvers")
    ax1.set_xticks(processors)
    ax1.grid(True, which="both", alpha=0.5)
    ax1.set_xlim(min(processors) - 1, max(processors) + 1)
    ax1.legend(loc="upper right", framealpha=0.9, fancybox=True)
    plt.tight_layout()

    if save_path_prefix:
        plt.savefig(f"{save_path_prefix}_total.pdf", dpi=300, bbox_inches="tight")
        print(f"Saved: {save_path_prefix}_total.pdf")
    plt.show()

    # Plot 2: Solve time
    fig2, ax2 = plt.subplots(figsize=(10, 7))
    for data in plot_data:
        ax2.plot(
            data["x"],
            data["y_solve"],
            marker=data["marker"],
            color=data["color"],
            linestyle=data["linestyle"],
            label=data["label"],
            markeredgecolor="black",
            markeredgewidth=0.5,
        )

    y_ref_solve = df["solve_time_avg"].max() * 0.5
    y_ideal_solve = get_ideal_scaling(x_ideal, y_ref_solve, x_ideal[0])
    ax2.plot(x_ideal, y_ideal_solve, "k--", linewidth=2, label="Ideal scaling")

    ax2.set_yscale("log")
    ax2.set_xlabel("Number of processors")
    ax2.set_ylabel("solve time (s)")
    ax2.set_title("Strong scaling for solve time for all the solvers")
    ax2.set_xticks(processors)
    ax2.grid(True, which="both", alpha=0.5)
    ax2.set_xlim(min(processors) - 1, max(processors) + 1)
    ax2.legend(loc="upper right", framealpha=0.9, fancybox=True)
    plt.tight_layout()

    if save_path_prefix:
        plt.savefig(f"{save_path_prefix}_solve.pdf", dpi=300, bbox_inches="tight")
        print(f"Saved: {save_path_prefix}_solve.pdf")
    plt.show()

    # Plot 3: Setup+Assemble time
    fig3, ax3 = plt.subplots(figsize=(10, 7))
    for data in plot_data:
        ax3.plot(
            data["x"],
            data["y_setup"],
            marker=data["marker"],
            color=data["color"],
            linestyle=data["linestyle"],
            label=data["label"],
            markeredgecolor="black",
            markeredgewidth=0.5,
        )

    y_ref_setup = (df["setup_time_avg"] + df["assembly_time_avg"]).max() * 0.3
    y_ideal_setup = get_ideal_scaling(x_ideal, y_ref_setup, x_ideal[0])
    ax3.plot(x_ideal, y_ideal_setup, "k--", linewidth=2, label="Ideal scaling")

    ax3.set_yscale("log")
    ax3.set_xlabel("Number of processors")
    ax3.set_ylabel("setup+assemble time (s)")
    ax3.set_title("Strong scaling for setup+assemble time for all the solvers")
    ax3.set_xticks(processors)
    ax3.grid(True, which="both", alpha=0.5)
    ax3.set_xlim(min(processors) - 1, max(processors) + 1)
    ax3.legend(loc="upper right", framealpha=0.9, fancybox=True)
    plt.tight_layout()

    if save_path_prefix:
        plt.savefig(f"{save_path_prefix}_setup.pdf", dpi=300, bbox_inches="tight")
        print(f"Saved: {save_path_prefix}_setup.pdf")
    plt.show()


# Generate the plots
plot_strong_scaling_all_solvers(pure_mpi_strong, "./report/figures/strong_scaling")

In [ ]:
def plot_solver_comparison(df, save_path=None):
    """
    Create side-by-side plots comparing matrix-free vs matrix-based solvers
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Get unique DoF values sorted descending
    dof_values = sorted(df["n_dofs"].unique(), reverse=True)
    processors = sorted(df["n_mpi"].unique())

    # Plot Matrix-Free
    mf_df = df[df["solver_type"] == "matrix_free"]
    for i, n_dofs in enumerate(dof_values):
        subset = mf_df[mf_df["n_dofs"] == n_dofs].sort_values("n_mpi")
        if len(subset) < 2:
            continue
        x = subset["n_mpi"].values
        y = subset["total_time_avg"].values
        color = COLOR_CYCLE[i % len(COLOR_CYCLE)]
        marker = MARKERS[i % len(MARKERS)]
        label = f"mf ({format_dofs(n_dofs)} DoFs)"
        ax1.plot(
            x,
            y,
            marker=marker,
            color=color,
            linestyle="-.",
            label=label,
            markeredgecolor="black",
            markeredgewidth=0.5,
        )

    # Plot Matrix-Based
    mb_df = df[df["solver_type"] == "matrix_based"]
    for i, n_dofs in enumerate(dof_values):
        subset = mb_df[mb_df["n_dofs"] == n_dofs].sort_values("n_mpi")
        if len(subset) < 2:
            continue
        x = subset["n_mpi"].values
        y = subset["total_time_avg"].values
        color = COLOR_CYCLE[i % len(COLOR_CYCLE)]
        marker = MARKERS[i % len(MARKERS)]
        label = f"mb ({format_dofs(n_dofs)} DoFs)"
        ax2.plot(
            x,
            y,
            marker=marker,
            color=color,
            linestyle="-.",
            label=label,
            markeredgecolor="black",
            markeredgewidth=0.5,
        )

    # Add ideal scaling lines
    x_ideal = np.array(processors)

    # Matrix-free ideal scaling
    mf_max = mf_df["total_time_avg"].max() * 0.7
    y_ideal_mf = get_ideal_scaling(x_ideal, mf_max, x_ideal[0])
    ax1.plot(x_ideal, y_ideal_mf, "k--", linewidth=2, label="Ideal scaling")

    # Matrix-based ideal scaling
    mb_max = mb_df["total_time_avg"].max() * 0.7
    y_ideal_mb = get_ideal_scaling(x_ideal, mb_max, x_ideal[0])
    ax2.plot(x_ideal, y_ideal_mb, "k--", linewidth=2, label="Ideal scaling")

    # Configure axes
    for ax in [ax1, ax2]:
        ax.set_yscale("log")
        ax.set_xlabel("Number of processors")
        ax.set_ylabel("total time (s)")
        ax.set_xticks(processors)
        ax.grid(True, which="both", alpha=0.5)
        ax.legend(loc="upper right", framealpha=0.9, fancybox=True)
        ax.set_xlim(min(processors) - 1, max(processors) + 1)

    ax1.set_title("Strong scaling of total time for mf")
    ax2.set_title("Strong scaling of total time for mb")

    # Add subplot labels below plots
    fig.text(0.25, 0.02, "(a) Matrixfree", ha="center", fontsize=12, fontweight="bold")
    fig.text(0.75, 0.02, "(b) Matrixbased", ha="center", fontsize=12, fontweight="bold")

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.12)

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches="tight")
        print(f"Saved: {save_path}")

    plt.show()


# Generate the plot
plot_solver_comparison(pure_mpi_strong, "solver_comparison.pdf")

In [ ]:
def plot_efficiency(df, save_path_prefix=None):
    """Plot parallel efficiency for all configurations"""
    dof_values = sorted(df["n_dofs"].unique(), reverse=True)
    processors = sorted(df["n_mpi"].unique())

    # Plot 1: Matrix-Free efficiency
    fig1, ax1 = plt.subplots(figsize=(10, 7))

    mf_df = df[df["solver_type"] == "matrix_free"]
    for i, n_dofs in enumerate(dof_values):
        subset = mf_df[mf_df["n_dofs"] == n_dofs].sort_values("n_mpi")
        if len(subset) < 2:
            continue
        x = subset["n_mpi"].values

        # Calculate speedup and efficiency
        base_time = subset["total_time_avg"].iloc[0]
        speedup = base_time / subset["total_time_avg"].values
        efficiency = speedup / (x / x[0])

        color = COLOR_CYCLE[i % len(COLOR_CYCLE)]
        marker = MARKERS[i % len(MARKERS)]
        label = f"mf ({format_dofs(n_dofs)} DoFs)"
        ax1.plot(
            x,
            efficiency * 100,
            marker=marker,
            color=color,
            linestyle="-.",
            label=label,
            markeredgecolor="black",
            markeredgewidth=0.5,
        )

    ax1.axhline(y=100, color="k", linestyle="--", linewidth=2, label="Ideal (100%)")
    ax1.set_xlabel("Number of processors")
    ax1.set_ylabel("Parallel Efficiency (%)")
    ax1.set_title("Parallel efficiency for Matrix-Free")
    ax1.set_xticks(processors)
    ax1.grid(True, alpha=0.5)
    ax1.legend(loc="upper right", framealpha=0.9, fancybox=True)
    ax1.set_xlim(min(processors) - 1, max(processors) + 1)
    ax1.set_ylim(0, 120)
    plt.tight_layout()

    if save_path_prefix:
        plt.savefig(f"{save_path_prefix}_mf.pdf", dpi=300, bbox_inches="tight")
        print(f"Saved: {save_path_prefix}_mf.pdf")
    plt.show()

    # Plot 2: Matrix-Based efficiency
    fig2, ax2 = plt.subplots(figsize=(10, 7))

    mb_df = df[df["solver_type"] == "matrix_based"]
    for i, n_dofs in enumerate(dof_values):
        subset = mb_df[mb_df["n_dofs"] == n_dofs].sort_values("n_mpi")
        if len(subset) < 2:
            continue
        x = subset["n_mpi"].values

        base_time = subset["total_time_avg"].iloc[0]
        speedup = base_time / subset["total_time_avg"].values
        efficiency = speedup / (x / x[0])

        color = COLOR_CYCLE[i % len(COLOR_CYCLE)]
        marker = MARKERS[i % len(MARKERS)]
        label = f"mb ({format_dofs(n_dofs)} DoFs)"
        ax2.plot(
            x,
            efficiency * 100,
            marker=marker,
            color=color,
            linestyle="-.",
            label=label,
            markeredgecolor="black",
            markeredgewidth=0.5,
        )

    ax2.axhline(y=100, color="k", linestyle="--", linewidth=2, label="Ideal (100%)")
    ax2.set_xlabel("Number of processors")
    ax2.set_ylabel("Parallel Efficiency (%)")
    ax2.set_title("Parallel efficiency for Matrix-Based")
    ax2.set_xticks(processors)
    ax2.grid(True, alpha=0.5)
    ax2.legend(loc="upper right", framealpha=0.9, fancybox=True)
    ax2.set_xlim(min(processors) - 1, max(processors) + 1)
    ax2.set_ylim(0, 120)
    plt.tight_layout()

    if save_path_prefix:
        plt.savefig(f"{save_path_prefix}_mb.pdf", dpi=300, bbox_inches="tight")
        print(f"Saved: {save_path_prefix}_mb.pdf")
    plt.show()


# Generate the plots
plot_efficiency(pure_mpi_strong, "./report/figures/parallel_efficiency")

In [ ]:
# Print summary statistics for the data
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

for solver in ["matrix_free", "matrix_based"]:
    solver_df = pure_mpi_strong[pure_mpi_strong["solver_type"] == solver]
    print(f"\n{solver.upper().replace('_', ' ')}:")
    print("-" * 40)

    for n_dofs in sorted(solver_df["n_dofs"].unique(), reverse=True):
        subset = solver_df[solver_df["n_dofs"] == n_dofs].sort_values("n_mpi")
        if len(subset) < 2:
            continue

        min_procs = subset["n_mpi"].min()
        max_procs = subset["n_mpi"].max()

        t_min = subset[subset["n_mpi"] == min_procs]["total_time_avg"].values[0]
        t_max = subset[subset["n_mpi"] == max_procs]["total_time_avg"].values[0]

        actual_speedup = t_min / t_max
        ideal_speedup = max_procs / min_procs
        efficiency = actual_speedup / ideal_speedup * 100

        print(f"  {format_dofs(n_dofs)} DoFs:")
        print(f"    Time: {t_min:.2f}s ({min_procs}p) -> {t_max:.2f}s ({max_procs}p)")
        print(f"    Speedup: {actual_speedup:.2f}x (ideal: {ideal_speedup:.1f}x)")
        print(f"    Efficiency: {efficiency:.1f}%")

In [ ]:
plt.rcParams.update(
    {
        "font.size": 10,
        "axes.grid": True,
        "grid.alpha": 0.5,
    }
)
# Load data
hybrid = pd.read_csv("result/combined_hybrid.csv")
pure_mpi = pd.read_csv("result/combined_pure_mpi.csv")
pure_thread = pd.read_csv("result/combined_pure_thread.csv")

n_dofs_target = 16785409

print("=" * 70)
print("ANALYSIS: Why Pure MPI Significantly Outperforms Pure Threading")
print("=" * 70)

# Compare breakdown of time components
print("\n1. TIME BREAKDOWN COMPARISON (28 cores, 16.8M DoFs)")
print("-" * 70)

for solver in ["matrix_free", "matrix_based"]:
    print(f"\n{solver.upper().replace('_', ' ')}:")

    # Pure MPI 28 cores
    mpi_28 = pure_mpi[
        (pure_mpi["solver_type"] == solver)
        & (pure_mpi["n_mpi"] == 28)
        & (pure_mpi["n_dofs"] == n_dofs_target)
    ]

    # Pure Thread 28 cores
    thread_28 = pure_thread[
        (pure_thread["solver_type"] == solver)
        & (pure_thread["n_threads"] == 28)
        & (pure_thread["n_dofs"] == n_dofs_target)
    ]

    if len(mpi_28) > 0 and len(thread_28) > 0:
        mpi = mpi_28.iloc[0]
        thread = thread_28.iloc[0]

        print(f"  {'Component':<20} {'Pure MPI':>12} {'Pure Thread':>12} {'Ratio':>10}")
        print(f"  {'-' * 54}")
        print(
            f"  {'Setup time':<20} {mpi['setup_time_avg']:>10.2f}s {thread['setup_time_avg']:>10.2f}s {thread['setup_time_avg'] / mpi['setup_time_avg']:>10.1f}x"
        )
        print(
            f"  {'Assembly time':<20} {mpi['assembly_time_avg']:>10.2f}s {thread['assembly_time_avg']:>10.2f}s {thread['assembly_time_avg'] / mpi['assembly_time_avg']:>10.1f}x"
        )
        print(
            f"  {'Solve time':<20} {mpi['solve_time_avg']:>10.2f}s {thread['solve_time_avg']:>10.2f}s {thread['solve_time_avg'] / mpi['solve_time_avg']:>10.1f}x"
        )
        print(
            f"  {'TOTAL time':<20} {mpi['total_time_avg']:>10.2f}s {thread['total_time_avg']:>10.2f}s {thread['total_time_avg'] / mpi['total_time_avg']:>10.1f}x"
        )
        print(
            f"  {'DoFs/second':<20} {mpi['dofs_per_second']:>10.0f} {thread['dofs_per_second']:>10.0f} {mpi['dofs_per_second'] / thread['dofs_per_second']:>10.1f}x"
        )

# Memory per process analysis
print("\n\n2. MEMORY LOCALITY ANALYSIS")
print("-" * 70)
print(f"  Problem size: {n_dofs_target:,} DoFs")
print("  ")
print("  Pure Threading (1 MPI × 28 threads):")
print(f"    - ALL {n_dofs_target:,} DoFs in single memory space")
print("    - 28 threads competing for same memory bandwidth")
print("    - High probability of remote NUMA access")
print("  ")
print("  Pure MPI (28 MPI × 1 thread):")
print(f"    - {n_dofs_target // 28:,} DoFs per process (local)")
print("    - Each process has dedicated memory bandwidth")
print("    - Optimal NUMA placement possible")

# Scaling efficiency analysis
print("\n\n3. SCALING EFFICIENCY COMPARISON")
print("-" * 70)

for solver in ["matrix_free", "matrix_based"]:
    print(f"\n{solver.upper().replace('_', ' ')}:")

    # Get baseline (1 core)
    mpi_1 = pure_mpi[
        (pure_mpi["solver_type"] == solver)
        & (pure_mpi["n_mpi"] == 1)
        & (pure_mpi["n_dofs"] == n_dofs_target)
    ]

    thread_1 = pure_thread[
        (pure_thread["solver_type"] == solver)
        & (pure_thread["n_threads"] == 1)
        & (pure_thread["n_dofs"] == n_dofs_target)
    ]

    if len(mpi_1) == 0:
        continue

    baseline = mpi_1.iloc[0]["total_time_avg"]

    print(
        f"  {'Cores':<8} {'MPI Time':>10} {'MPI Eff':>10} {'Thread Time':>12} {'Thread Eff':>12}"
    )
    print(f"  {'-' * 52}")

    for n_cores in [1, 2, 4, 7, 14, 28]:
        mpi_data = pure_mpi[
            (pure_mpi["solver_type"] == solver)
            & (pure_mpi["n_mpi"] == n_cores)
            & (pure_mpi["n_dofs"] == n_dofs_target)
        ]
        thread_data = pure_thread[
            (pure_thread["solver_type"] == solver)
            & (pure_thread["n_threads"] == n_cores)
            & (pure_thread["n_dofs"] == n_dofs_target)
        ]

        mpi_time = (
            mpi_data.iloc[0]["total_time_avg"] if len(mpi_data) > 0 else float("nan")
        )
        thread_time = (
            thread_data.iloc[0]["total_time_avg"]
            if len(thread_data) > 0
            else float("nan")
        )

        mpi_eff = (
            (baseline / mpi_time / n_cores * 100)
            if not np.isnan(mpi_time)
            else float("nan")
        )
        thread_eff = (
            (baseline / thread_time / n_cores * 100)
            if not np.isnan(thread_time)
            else float("nan")
        )

        mpi_str = f"{mpi_time:.1f}s" if not np.isnan(mpi_time) else "N/A"
        thread_str = f"{thread_time:.1f}s" if not np.isnan(thread_time) else "N/A"
        mpi_eff_str = f"{mpi_eff:.1f}%" if not np.isnan(mpi_eff) else "N/A"
        thread_eff_str = f"{thread_eff:.1f}%" if not np.isnan(thread_eff) else "N/A"

        print(
            f"  {n_cores:<8} {mpi_str:>10} {mpi_eff_str:>10} {thread_str:>12} {thread_eff_str:>12}"
        )

# ==================== FIGURE 1: Time Breakdown ====================
fig1, ax1 = plt.subplots(figsize=(10, 7))

for i, solver in enumerate(["matrix_free", "matrix_based"]):
    mpi_28 = pure_mpi[
        (pure_mpi["solver_type"] == solver)
        & (pure_mpi["n_mpi"] == 28)
        & (pure_mpi["n_dofs"] == n_dofs_target)
    ]
    thread_28 = pure_thread[
        (pure_thread["solver_type"] == solver)
        & (pure_thread["n_threads"] == 28)
        & (pure_thread["n_dofs"] == n_dofs_target)
    ]

    if len(mpi_28) > 0 and len(thread_28) > 0:
        x = [i * 3, i * 3 + 1]
        setup = [mpi_28.iloc[0]["setup_time_avg"], thread_28.iloc[0]["setup_time_avg"]]
        assembly = [
            mpi_28.iloc[0]["assembly_time_avg"],
            thread_28.iloc[0]["assembly_time_avg"],
        ]
        solve = [mpi_28.iloc[0]["solve_time_avg"], thread_28.iloc[0]["solve_time_avg"]]

        ax1.bar(x, setup, width=0.8, label="Setup" if i == 0 else "", color="#2ecc71")
        ax1.bar(
            x,
            assembly,
            width=0.8,
            bottom=setup,
            label="Assembly" if i == 0 else "",
            color="#3498db",
        )
        ax1.bar(
            x,
            solve,
            width=0.8,
            bottom=[s + a for s, a in zip(setup, assembly)],
            label="Solve" if i == 0 else "",
            color="#e74c3c",
        )

ax1.set_xticks([0, 1, 3, 4])
ax1.set_xticklabels(["MPI\n(MF)", "Thread\n(MF)", "MPI\n(MB)", "Thread\n(MB)"])
ax1.set_ylabel("Time (s)")
ax1.set_title("Time Breakdown: Pure MPI vs Pure Threading (28 cores)")
ax1.legend()
plt.tight_layout()
plt.savefig(
    "./report/figures/mpi_vs_thread_analysis_breakdown.pdf",
    dpi=300,
    bbox_inches="tight",
)
print("Saved: mpi_vs_thread_analysis_breakdown.pdf")
plt.show()

# ==================== FIGURE 2: Performance Ratio ====================
fig2, ax2 = plt.subplots(figsize=(10, 7))

cores = [2, 4, 7, 14, 28]
for solver, color, marker in [
    ("matrix_free", "#1f77b4", "o"),
    ("matrix_based", "#ff7f0e", "s"),
]:
    ratios = []
    for n_cores in cores:
        mpi_data = pure_mpi[
            (pure_mpi["solver_type"] == solver)
            & (pure_mpi["n_mpi"] == n_cores)
            & (pure_mpi["n_dofs"] == n_dofs_target)
        ]
        thread_data = pure_thread[
            (pure_thread["solver_type"] == solver)
            & (pure_thread["n_threads"] == n_cores)
            & (pure_thread["n_dofs"] == n_dofs_target)
        ]
        if len(mpi_data) > 0 and len(thread_data) > 0:
            ratio = (
                thread_data.iloc[0]["total_time_avg"]
                / mpi_data.iloc[0]["total_time_avg"]
            )
            ratios.append(ratio)
        else:
            ratios.append(np.nan)

    ax2.plot(
        cores,
        ratios,
        marker=marker,
        color=color,
        linewidth=2,
        markersize=8,
        label=solver.replace("_", " ").title(),
        markeredgecolor="black",
    )

ax2.axhline(y=1, color="k", linestyle="--", alpha=0.5, label="Equal performance")
ax2.set_xlabel("Number of cores")
ax2.set_ylabel("Thread Time / MPI Time")
ax2.set_title("Performance Ratio")
ax2.legend()
ax2.set_xticks(cores)
ax2.set_ylim(0, max([r for r in ratios if not np.isnan(r)]) * 2 if ratios else 50)
ax2.grid(True, alpha=0.5)
plt.tight_layout()
plt.savefig(
    "./report/figures/mpi_vs_thread_analysis_ratio.pdf", dpi=300, bbox_inches="tight"
)
print("Saved: mpi_vs_thread_analysis_ratio.pdf")
plt.show()

# ==================== FIGURE 3: Memory Locality ====================
fig3, ax3 = plt.subplots(figsize=(10, 7))

configs = [
    "1 MPI\n(thread=28)",
    "2 MPI\n(thread=14)",
    "4 MPI\n(thread=7)",
    "7 MPI\n(thread=4)",
    "14 MPI\n(thread=2)",
    "28 MPI\n(thread=1)",
]
dofs_per_proc = [
    n_dofs_target / 1,
    n_dofs_target / 2,
    n_dofs_target / 4,
    n_dofs_target / 7,
    n_dofs_target / 14,
    n_dofs_target / 28,
]

colors_map = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(configs)))
bars = ax3.bar(
    range(len(configs)),
    [d / 1e6 for d in dofs_per_proc],
    color=colors_map,
    edgecolor="black",
)
ax3.set_xticks(range(len(configs)))
ax3.set_xticklabels(configs, rotation=45, ha="right")
ax3.set_ylabel("DoFs per MPI Process (Millions)")
ax3.set_title("Memory Locality: DoFs per Process")

# Add text annotations
for bar, dof in zip(bars, dofs_per_proc):
    ax3.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.3,
        f"{dof / 1e6:.1f}M",
        ha="center",
        fontsize=9,
    )

plt.tight_layout()
plt.savefig(
    "./report/figures/mpi_vs_thread_analysis_memory.pdf", dpi=300, bbox_inches="tight"
)
print("Saved: mpi_vs_thread_analysis_memory.pdf")
plt.show()

# ==================== FIGURE 4: Parallel Efficiency ====================
fig4, ax4 = plt.subplots(figsize=(10, 7))

for solver, color, marker in [
    ("matrix_free", "#1f77b4", "o"),
    ("matrix_based", "#ff7f0e", "s"),
]:
    mpi_1 = pure_mpi[
        (pure_mpi["solver_type"] == solver)
        & (pure_mpi["n_mpi"] == 1)
        & (pure_mpi["n_dofs"] == n_dofs_target)
    ]
    if len(mpi_1) == 0:
        continue
    baseline = mpi_1.iloc[0]["total_time_avg"]

    # MPI efficiency
    mpi_eff = []
    thread_eff = []
    valid_cores = []

    for n_cores in [1, 2, 4, 7, 14, 28]:
        mpi_data = pure_mpi[
            (pure_mpi["solver_type"] == solver)
            & (pure_mpi["n_mpi"] == n_cores)
            & (pure_mpi["n_dofs"] == n_dofs_target)
        ]
        thread_data = pure_thread[
            (pure_thread["solver_type"] == solver)
            & (pure_thread["n_threads"] == n_cores)
            & (pure_thread["n_dofs"] == n_dofs_target)
        ]

        if len(mpi_data) > 0:
            mpi_eff.append(
                baseline / mpi_data.iloc[0]["total_time_avg"] / n_cores * 100
            )
        if len(thread_data) > 0:
            thread_eff.append(
                baseline / thread_data.iloc[0]["total_time_avg"] / n_cores * 100
            )
            valid_cores.append(n_cores)

    cores_mpi = [1, 2, 4, 7, 14, 28][: len(mpi_eff)]
    ax4.plot(
        cores_mpi,
        mpi_eff,
        marker=marker,
        color=color,
        linewidth=2,
        label=f"{solver.replace('_', ' ').title()} MPI",
        markeredgecolor="black",
    )
    ax4.plot(
        valid_cores,
        thread_eff,
        marker=marker,
        color=color,
        linewidth=2,
        linestyle="--",
        label=f"{solver.replace('_', ' ').title()} Thread",
        markeredgecolor="black",
        alpha=0.6,
    )

ax4.axhline(y=100, color="k", linestyle=":", alpha=0.5, label="Ideal (100%)")
ax4.set_xlabel("Number of cores")
ax4.set_ylabel("Parallel Efficiency (%)")
ax4.set_title("Parallel Efficiency: MPI vs Threading")
ax4.legend(loc="upper right", fontsize=10)
ax4.set_xticks([1, 2, 4, 7, 14, 28])
ax4.set_ylim(0, 120)
ax4.grid(True, alpha=0.5)
plt.tight_layout()
plt.savefig(
    "./report/figures/mpi_vs_thread_analysis_efficiency.pdf",
    dpi=300,
    bbox_inches="tight",
)
print("Saved: mpi_vs_thread_analysis_efficiency.pdf")
plt.show()

# Summary
print("\n\n" + "=" * 70)
print("SUMMARY: Why Pure MPI >> Pure Threading")
print("=" * 70)
print("""
1. MEMORY BANDWIDTH SATURATION
   - Threading: 28 threads share ONE memory controller
   - MPI: 28 processes can use MULTIPLE memory controllers
   
2. NUMA (Non-Uniform Memory Access) EFFECTS
   - Threading: Threads on remote NUMA nodes = 3-4x memory latency
   - MPI: Each process binds to local NUMA node = optimal latency
   
3. CACHE EFFICIENCY
   - Threading: 16.8M DoFs don't fit in shared L3 cache
   - MPI: 600K DoFs per process fits better in local cache
   
4. FALSE SHARING
   - Threading: Adjacent cache lines invalidated between threads
   - MPI: No shared memory = no false sharing
   
5. SYNCHRONIZATION OVERHEAD
   - Threading: Barriers, memory fences at every parallel region
   - MPI: Communication only at domain boundaries
   
6. LIBRARY DESIGN
   - deal.II, PETSc, Trilinos optimized for distributed memory (MPI)
   - Thread parallelism is secondary in most HPC libraries
""")